In [1]:
import pandas as pd

wget needs to be installed with brew install wget, not pip install

In [5]:
# Download the file first
!wget https://raw.githubusercontent.com/jinshijian/SoilHealthDB/master/data/SoilHealthDB_V2.csv

# Read the downloaded CSV file
df = pd.read_csv('./data/SoilHealthDB.csv', encoding='latin-1')

# Display the first few rows to verify
print(df.head())

--2025-06-20 10:32:32--  https://raw.githubusercontent.com/jinshijian/SoilHealthDB/master/data/SoilHealthDB_V2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3545298 (3.4M) [application/octet-stream]
Saving to: ‘SoilHealthDB_V2.csv.1’

SoilHealthDB_V2.csv 100%[===================>]   3.38M  --.-KB/s    in 0.05s   

2025-06-20 10:32:32 (63.1 MB/s) - ‘SoilHealthDB_V2.csv.1’ saved [3545298/3545298]

   ï»¿StudyID ExperimentID Author_F Author_G  YearPublication  SamplingYear  \
0           1          1-1    Abawi     G.S.             2000           NaN   
1           1          1-1    Abawi     G.S.             2000           NaN   
2           1          1-2    Abawi     G.S.             2000           NaN   
3           1          1-1    Abawi     G.S.   

/var/folders/1j/14pr84p97pn6v_px_2rn8l_00000gn/T/ipykernel_67355/4172277652.py:7: DtypeWarning: Columns (16,19,20,52,77,122,150,152,160,192,197,212,232,237,242,247,252,262,265,266) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/SoilHealthDB.csv', encoding='latin-1')


## cleaning

Remove all columns containing 'comment'

In [6]:
# prompt: remove all columns containing 'comment'

columns_to_drop = [col for col in df.columns if 'comment' in col.lower()]
df_cleaned = df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned DataFrame
print(df_cleaned.head())

   ï»¿StudyID ExperimentID Author_F Author_G  YearPublication  SamplingYear  \
0           1          1-1    Abawi     G.S.             2000           NaN   
1           1          1-1    Abawi     G.S.             2000           NaN   
2           1          1-2    Abawi     G.S.             2000           NaN   
3           1          1-1    Abawi     G.S.             2000           NaN   
4           1          1-4    Abawi     G.S.             2000           NaN   

                Journal SiteInfor Country  Latitude  ...  MBN_T  MBN_C_SD  \
0  Applied Soil Ecology  New York     USA     40.71  ...    NaN       NaN   
1  Applied Soil Ecology  New York     USA     40.71  ...    NaN       NaN   
2  Applied Soil Ecology  New York     USA     40.71  ...    NaN       NaN   
3  Applied Soil Ecology  New York     USA     40.71  ...    NaN       NaN   
4  Applied Soil Ecology  New York     USA     40.71  ...    NaN       NaN   

   MBN_T_SD  Microelement  SQI  ESS Texture_C  Unnamed: 270  U

Remove records that don't have organic carbon data

In [7]:
# prompt: remove all records that do not contain values for OC__T or OC_C

df_filtered = df.dropna(subset=['OC_T', 'OC_C'])

print(df_filtered.head())
print(f"Original number of rows: {len(df)}")
print(f"Number of rows after filtering: {len(df_filtered)}")


    ï»¿StudyID ExperimentID       Author_F  Author_G  YearPublication  \
11           2          2-1         Bandic      Anna             1999   
12           2          2-2         Bandic      Anna             1999   
55           4          4-1  Blanco-Canqui  Humberto             2010   
56           4          4-2  Blanco-Canqui  Humberto             2010   
57           4          4-1  Blanco-Canqui  Humberto             2010   

    SamplingYear                                     Journal  \
11        1994.0               Soil Biology and Biochemistry   
12        1994.0               Soil Biology and Biochemistry   
55        2002.0  Soil and Water Management and Conservation   
56        2002.0  Soil and Water Management and Conservation   
57        2002.0  Soil and Water Management and Conservation   

                                            SiteInfor Country  Latitude  ...  \
11  North  Willamette  Research  and Extension  Ce...     USA     45.23  ...   
12  North  Willa

Make new df with only fields that will be input by user about their property (as these are the only ones that can be used to predict the soil carbon of their property).  temperature, precipitation, cover crop (does it use cover crop), landuse type, type of tillage, crop, Elevation. Make sure all are numeric (i.e. use onehotencoder). Not study found texture is largest indicator, but most records don't have texture recorded. SoilFamily might be a substitute? Need to use ControlDescription - combine with cover crop field to indicate whether cover crop or not. i.e. fallow, lupin etc. in one field

Fields:
- Tannual : annual temperature
- Pannual : annual precipitation


Fields to be created or used with onehotencoder
- Fertilization _C _T : removed as incomplete and messy
- Tillage _C _T
- CoverCropGroup (LG is legumes, needs to be cleaned)
- GrainCropGroup
- Conservation_Type

In [8]:
# prompt: create new df with fields Tannual, Pannual, Fertilization_C, Fertilization_T, Tillage_C, Tillage_T, CoverCropGroup, GrainCrop, Conservation_Type

# Select the desired columns to create the new DataFrame
df_new = df_filtered[['Country', 'Tannual', 'Pannual', 'OC_T', 'OC_C', 'Tillage_C', 'Tillage_T', 'CoverCropGroup', 'GrainCropGroup', 'Conservation_Type']].copy()

print(df_new.head())

   Country  Tannual  Pannual   OC_T   OC_C   Tillage_C   Tillage_T  \
11     USA      NaN      NaN  3.430  3.930         NaN         NaN   
12     USA      NaN      NaN  4.130  3.930         NaN         NaN   
55     USA      NaN      NaN  1.693  1.368  No tillage  No tillage   
56     USA      NaN      NaN  1.071  1.031  No tillage  No tillage   
57     USA      NaN      NaN  1.816  1.368  No tillage  No tillage   

   CoverCropGroup GrainCropGroup Conservation_Type  
11            Rye            MTT                CC  
12         Legume            MTT                CC  
55         Legume             WO             CC-NT  
56         Legume             WO             CC-NT  
57         Legume             WO             CC-NT  


In [9]:
# prompt: check how many rows df_new has

print(f"Number of rows in df_new: {len(df_new)}")

Number of rows in df_new: 1573


In [ ]:
# check how many na's df_new has for Tannual and Pannual

print(f"Number of NA's in Tannual column: {df_new['Tannual'].isnull().sum()}")
print(f"Number of NA's in Pannual column: {df_new['Pannual'].isnull().sum()}")


Number of NA's in Tannual column: 1156
Number of NA's in Pannual column: 1095


In [11]:
# remove records where Tannual is null

df_new_cleaned = df_new.dropna(subset=['Tannual'])

print(df_new_cleaned.head())
print(f"Number of rows after removing null Tannual: {len(df_new_cleaned)}")
print(f"Number of NA's in Tannual column after cleaning: {df_new_cleaned['Tannual'].isnull().sum()}")

    Country  Tannual  Pannual  OC_T  OC_C             Tillage_C  \
424     USA     19.6    976.0  0.74  0.46  Conventional tillage   
425     USA     19.6    976.0  0.70  0.46  Conventional tillage   
426     USA     19.6    976.0  0.59  0.46  Conventional tillage   
427     USA     19.6    976.0  0.73  0.50  Conventional tillage   
428     USA     19.6    976.0  0.74  0.50  Conventional tillage   

                Tillage_T CoverCropGroup GrainCropGroup Conservation_Type  
424  Conservation tillage             LL           Corn                CC  
425  Conservation tillage         Legume           Corn                CC  
426  Conservation tillage         Legume           Corn                CC  
427  Conservation tillage             LL           Corn                CC  
428  Conservation tillage         Legume           Corn                CC  
Number of rows after removing null Tannual: 417
Number of NA's in Tannual column after cleaning: 0


In [12]:
print(f"Number of NA's in Tannual column: {df_new_cleaned['Tannual'].isnull().sum()}")
print(f"Number of NA's in Pannual column: {df_new_cleaned['Pannual'].isnull().sum()}")
print(f"Number of NA's in Pannual column: {df_new_cleaned['CoverCropGroup'].isnull().sum()}")


Number of NA's in Tannual column: 0
Number of NA's in Pannual column: 3
Number of NA's in Pannual column: 3


In [13]:
# drop the records with Pannual of null

df_new_cleaned = df_new_cleaned.dropna(subset=['Pannual'])

print(df_new_cleaned.head())
print(f"Number of rows after removing null Pannual: {len(df_new_cleaned)}")
print(f"Number of NA's in Pannual column after cleaning: {df_new_cleaned['Pannual'].isnull().sum()}")

    Country  Tannual  Pannual  OC_T  OC_C             Tillage_C  \
424     USA     19.6    976.0  0.74  0.46  Conventional tillage   
425     USA     19.6    976.0  0.70  0.46  Conventional tillage   
426     USA     19.6    976.0  0.59  0.46  Conventional tillage   
427     USA     19.6    976.0  0.73  0.50  Conventional tillage   
428     USA     19.6    976.0  0.74  0.50  Conventional tillage   

                Tillage_T CoverCropGroup GrainCropGroup Conservation_Type  
424  Conservation tillage             LL           Corn                CC  
425  Conservation tillage         Legume           Corn                CC  
426  Conservation tillage         Legume           Corn                CC  
427  Conservation tillage             LL           Corn                CC  
428  Conservation tillage         Legume           Corn                CC  
Number of rows after removing null Pannual: 414
Number of NA's in Pannual column after cleaning: 0


In [14]:
# count conservation_Type

conservation_type_counts = df_new_cleaned['Conservation_Type'].value_counts()
print("Counts of each Conservation_Type:")
conservation_type_counts

Counts of each Conservation_Type:


Conservation_Type
CC       277
CC-NT    124
OF-CC     13
Name: count, dtype: int64

So much temperature and precipitation data is missing, test using country instead compared to using much lower amount of records with Tannual and Pannual data. Later version can get an estimate of these values from the lat/long to use more of the data.

In [15]:
# duplicate records to have all fields with '_T' in seperate records to '_C'. CoverCropGroup and Conservation_Type should be 'None' for all '_C' records

# Duplicate records for '_T' and '_C'
df_t = df_new_cleaned.copy()
df_c = df_new_cleaned.copy()

# Rename columns for '_T' records
df_t.rename(columns={'OC_T': 'OC',
                     'Tillage_T': 'Tillage'}, inplace=True)
df_t = df_t.drop(columns=['OC_C', 'Tillage_C'])
df_t['Type'] = '_T'


# Rename columns for '_C' records and set 'CoverCropGroup' and 'Conservation_Type' to None
df_c.rename(columns={'OC_C': 'OC',
                     'Tillage_C': 'Tillage'}, inplace=True)
df_c = df_c.drop(columns=['OC_T', 'Tillage_T'])
df_c['Type'] = '_C'
df_c['CoverCropGroup'] = 'None'
df_c['Conservation_Type'] = 'None'


# Concatenate the two dataframes
df_combined = pd.concat([df_t, df_c], ignore_index=True)

# Display the first few rows of the combined DataFrame
print(df_combined.head())
print(df_combined.tail())

# Check the number of rows in the combined DataFrame
print(f"Number of rows in combined DataFrame: {len(df_combined)}")
print(f"Number of rows in df_new_cleaned: {len(df_new_cleaned)}")

  Country  Tannual  Pannual    OC               Tillage CoverCropGroup  \
0     USA     19.6    976.0  0.74  Conservation tillage             LL   
1     USA     19.6    976.0  0.70  Conservation tillage         Legume   
2     USA     19.6    976.0  0.59  Conservation tillage         Legume   
3     USA     19.6    976.0  0.73  Conservation tillage             LL   
4     USA     19.6    976.0  0.74  Conservation tillage         Legume   

  GrainCropGroup Conservation_Type Type  
0           Corn                CC   _T  
1           Corn                CC   _T  
2           Corn                CC   _T  
3           Corn                CC   _T  
4           Corn                CC   _T  
    Country  Tannual  Pannual    OC              Tillage CoverCropGroup  \
823     USA     17.3   1958.0  0.87  Traditional tillage           None   
824     USA     17.3   1958.0  0.86  Traditional tillage           None   
825     USA     17.3   1958.0  0.90  Traditional tillage           None   
826

In [16]:
# list unique values of GrainCropGroup

print(df_combined['GrainCropGroup'].unique())

['Corn' 'Arable' 'Soybean' 'CS' 'Vegetable' 'CV' 'CO' 'MTT' 'CW' 'Wheat'
 'WO' 'AVG']


In [17]:
# list unique tillage types

print(df_combined['Tillage'].unique())

['Conservation tillage' 'no tillage' nan 'spring tillage'
 'conventional tillage' 'Strip tillage' 'Chisel tillage'
 'no tillage/chisel plowing/moldboard plowing mixed' 'chisel plowing'
 'moldboard plowing mixed' 'No tillage' 'CT' 'no data' 'No till'
 'No-tillage' 'Conservational tillage' 'Conventional tillage'
 'CT / NT (not clear which one, influence found to be not significant)'
 'Disc till' 'Harrowing' 'mouldboard ploughing' 'Traditional tillage']


In [18]:
print(f"Number of NA's in Tillage column: {df_combined['Tillage'].isnull().sum()}")

Number of NA's in Tillage column: 101


In [19]:
# prompt: check how many 'no data' records there are for Tillage

print(f"Number of 'no data' records for Tillage: {(df_combined['Tillage'] == 'no data').sum()}")

Number of 'no data' records for Tillage: 13


For Tillage, null values, and 'no data' can be changed to Unknown. Fertilization needs to be cleaned, but the data is confusing and incomplete, so I will try the analysis without this field.

In [20]:
# prompt: make all null and 'no data' records for Tillage as 'Unknown'

df_combined['Tillage'] = df_combined['Tillage'].fillna('Unknown')
df_combined['Tillage'] = df_combined['Tillage'].replace('no data', 'Unknown')

print(df_combined['Tillage'].unique())
print(f"Number of 'Unknown' records for Tillage after cleaning: {(df_combined['Tillage'] == 'Unknown').sum()}")

['Conservation tillage' 'no tillage' 'Unknown' 'spring tillage'
 'conventional tillage' 'Strip tillage' 'Chisel tillage'
 'no tillage/chisel plowing/moldboard plowing mixed' 'chisel plowing'
 'moldboard plowing mixed' 'No tillage' 'CT' 'No till' 'No-tillage'
 'Conservational tillage' 'Conventional tillage'
 'CT / NT (not clear which one, influence found to be not significant)'
 'Disc till' 'Harrowing' 'mouldboard ploughing' 'Traditional tillage']
Number of 'Unknown' records for Tillage after cleaning: 114


In [21]:
# prompt: Make 'no tillage', 'No tillage', 'No till', 'No-tillage', 'CT / NT', as ''None'

tillage_replacements = {
    'no tillage': 'None',
    'No tillage': 'None',
    'No till': 'None',
    'No-tillage': 'None',
    'CT / NT (not clear which one, influence found to be not significant)': 'None'
}

df_combined['Tillage'] = df_combined['Tillage'].replace(tillage_replacements)

print(df_combined['Tillage'].unique())


['Conservation tillage' 'None' 'Unknown' 'spring tillage'
 'conventional tillage' 'Strip tillage' 'Chisel tillage'
 'no tillage/chisel plowing/moldboard plowing mixed' 'chisel plowing'
 'moldboard plowing mixed' 'CT' 'Conservational tillage'
 'Conventional tillage' 'Disc till' 'Harrowing' 'mouldboard ploughing'
 'Traditional tillage']


In [27]:
conventional_replacements = {
    'spring tillage': 'Conventional',
    'conventional tillage': 'Conventional',
    'moldboard plowing mixed': 'Conventional',
    'CT': 'Conventional',
    'Conventional tillage':'Conventional',
    'Disc till':'Conventional',
    'Harrowing':'Conventional',
    'mouldboard ploughing':'Conventional',
    'Traditional tillage':'Conventional'
}

reduced_replacements = {
    'Conservation tillage': 'Reduced',
    'Strip tillage': 'Reduced',
    'Chisel tillage': 'Reduced',
    'no tillage/chisel plowing/moldboard plowing mixed': 'Reduced',
    'chisel plowing':'Reduced',
    'Conservational tillage':'Reduced'
}

df_combined['Tillage'] = df_combined['Tillage'].replace(conventional_replacements)
df_combined['Tillage'] = df_combined['Tillage'].replace(reduced_replacements)
df_combined['Tillage'] = df_combined['Tillage'].replace('None', 'No')

print(df_combined['Tillage'].unique())

['Reduced' 'No' 'Unknown' 'Conventional']


In [23]:
df_combined.head(5)

,Country,Tannual,Pannual,OC,Tillage,CoverCropGroup,GrainCropGroup,Conservation_Type,Type
0,USA,19.6,976.0,0.74,Reduced,LL,Corn,CC,_T
1,USA,19.6,976.0,0.70,Reduced,Legume,Corn,CC,_T
2,USA,19.6,976.0,0.59,Reduced,Legume,Corn,CC,_T
3,USA,19.6,976.0,0.73,Reduced,LL,Corn,CC,_T
4,USA,19.6,976.0,0.74,Reduced,Legume,Corn,CC,_T


In [24]:
print(df_combined['CoverCropGroup'].unique())

['LL' 'Legume' 'Rye' 'Grass' 'Brassica' 'LG' 'Mixed' 'BG' 'Not_available'
 'AVG' 'None']


LL is Legume;
LG, BG, and AVG are Mixed;
Not_available is Unknown (consistent with Tillage);
None becomes nan when reading in dataset, so change to 'No'

In [26]:
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace('LL', 'Legume')
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace(['LG', 'BG', 'AVG'], 'Mixed')
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace('Not_available', 'Unknown')
df_combined['CoverCropGroup'] = df_combined['CoverCropGroup'].replace('None', 'No')

print(df_combined['CoverCropGroup'].unique())


['Legume' 'Rye' 'Grass' 'Brassica' 'Mixed' 'Unknown' 'No']


In [28]:
print(df_combined['GrainCropGroup'].unique())

['Corn' 'Arable' 'Soybean' 'CS' 'Vegetable' 'CV' 'CO' 'MTT' 'CW' 'Wheat'
 'WO' 'AVG']


Monoculture: Corn, Soybean, Wheat, Arable
Mono-Rotation: CS, CO, MTT, CW, WO
Unknown: AVG


In [ ]:
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace(['Corn', 'Soybean', 'Wheat','Arable'], 'Grain')
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace(['CS', 'CO', 'MTT', 'CW', 'WO'], 'Grain-rotation')
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace('AVG', 'Unknown')
df_combined['GrainCropGroup'] = df_combined['GrainCropGroup'].replace('CV', 'Vegetable')


print(df_combined['GrainCropGroup'].unique())

['Monoculture' 'Monoculture-rotation' 'Vegetable' 'Unknown']


In [33]:
df_combined.to_csv('data/soc_dataset.csv', index=False)